In [1]:
# Random Forest Classification

# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Alphabet import IUPAC
from Bio.SeqRecord import SeqRecord
from Bio import SeqUtils
from sklearn.model_selection import cross_val_score

# Importing the dataset
dataset = pd.read_csv('final file hopefully no errors.csv')

#for fib2.cvs only

dataset.loc[dataset['Gen'] == 'ssRNA(+)', 'Gen'] = 1
dataset.loc[dataset['Gen'] == 'ssRNA(-)', 'Gen'] = 0

p = dataset.loc[dataset['Gen'] == 1].reset_index(drop = True)
n = dataset.loc[dataset['Gen'] == 0].reset_index(drop = True)

pn = pd.concat([p,n]).reset_index(drop = True)
res = pd.read_csv('res 340.csv')
r = res[['ATC', 'ATCA', 'ATCT', 'TGGT']].copy()
pnr = pd.concat([pn, r], axis = 1)
#pnr = pnr.drop(index = pnr.loc[pnr['MolTyp'] == 'DNA'].index)
pnr = pnr.drop(columns=['Id', 'MolTyp', 'Topol', 'Des'])
#pnr = pnr.rename(columns={'ATC' : 'E1', 'ATCA' : 'E2', 'ATCT' : 'E3', 'TGGT' : 'E4'})

def orf_translator(record, table, min_pro_len):
    s1 = []
    s2 = []
    for strand, nuc in [(+1, record.seq), (-1, record.seq.reverse_complement())]:
        for frame in range(3):
            length = 3 * ((len(record)-frame) // 3) #Multiple of three
            for pro in nuc[frame:frame+length].translate(table).split("*"):
                if len(pro) >= min_pro_len:
                    if strand == 1:
                        s1.append(pro)
                    else:
                        s2.append(pro)
    return s1,s2


def whole(pnr):
    
    #sampling
    ps = pnr.loc[pnr['Gen'] == 1]
    ns = pnr.loc[pnr['Gen'] == 0]

    pnrs = pd.concat([ps,ns])
    
    
    #feature 1: length
    pnrs['F1'] = [len(i) for i in pnrs['Seq'].to_list()]
    
    f2, f3, f4, f5, f6, f7 = [],[],[],[],[],[]

    for i in pnrs['Seq'].to_list():
        a = SeqRecord(Seq(i))
        (s1, s2) = orf_translator(a, 11, 15)
        f2.append(len(s1))
        f3.append(len(s2))

    pnrs['F2'] = f2
    pnrs['F3'] = f3
    
    #Splitting features and labels
    X = pnrs.iloc[:, 3:].values.astype(float)
    y = pnrs.iloc[:, 1].values.astype(float)
    
    # Splitting the dataset into the Training set and Test set
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)
    
    # Feature Scaling
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)


    
    # Fitting Random Forest Classification to the Training set
    from sklearn.ensemble import RandomForestClassifier
    classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
    classifier.fit(X_train, y_train)
    
    # Predicting the Test set results
    y_pred = classifier.predict(X_test)
    
    # Making the Confusion Matrix
    from sklearn.metrics import confusion_matrix
    cm = confusion_matrix(y_test, y_pred)

    scores_mac = cross_val_score(classifier, X, y, cv=5, scoring='f1_macro')
    
    return scores_mac  

In [2]:
aclist = []

for i in range(20):
    fmac = whole(pnr)
    aclist.append(fmac.mean())
    print(fmac.mean(), fmac.std() *2)

print(np.mean(aclist))

0.9624808475878766 0.0632894338450859
0.9549906220689384 0.04359149716868421
0.957402241862148 0.03422942331248438
0.9449898392308601 0.046371298778821075
0.9699718118550008 0.04363901120221446
0.9649859220981686 0.010014143933072362
0.9549679285330752 0.04901410130189069
0.9574730162579119 0.0300459345683051
0.9674384652345042 0.037578507350368974
0.9674933568712513 0.012245535152485122
0.9524738049668274 0.0186961766053732
0.9549577177825013 0.04369296351364044
0.9474636045160845 0.01869635785656151
0.957483974542223 0.04362020188275059
0.9649459663602808 0.06614917334063593
0.9399107635376021 0.05346588285945927
0.9574839686679452 0.019993768891602367
0.952491012754983 0.04848147128319739
0.9574964838255978 0.03391418522168333
0.9449365957970726 0.034014077987756755
0.9565918972175427


In [ ]:
#using ['GTGG', 'TGGC', 'TGGT', 'CGTG', 'ATC', 'ATCA']

In [9]:
r = res[['GTGG', 'TGGC', 'TGGT', 'CGTG', 'ATC', 'ATCA']].copy()
pnr = pd.concat([pn, r], axis = 1)
pnr = pnr.drop(index = pnr.loc[pnr['MolTyp'] == 'DNA'].index)
pnr = pnr.drop(columns=['Id', 'MolTyp', 'Topol', 'Des'])
pnr = pnr.rename(columns={'GTGG' : 'E1', 'TGGC' : 'E2', 'TGGT' : 'E3', 'CGTG' : 'E4', 'ATC' : 'E5', 'ATCA' : 'E6'})

def whole(pnr):

    ps = pnr.loc[pnr['Gen'] == 1].sample(200)
    ns = pnr.loc[pnr['Gen'] == 0].sample(200)

    pnrs = pd.concat([ps,ns])
    
    
    #feature 1: length
    pnrs['F1'] = [len(i) for i in pnrs['Seq'].to_list()]
    
    f2, f3, f4, f5, f6, f7 = [],[],[],[],[],[]

    for i in pnrs['Seq'].to_list():
        a = SeqRecord(Seq(i))
        (s1, s2) = orf_translator(a, 11, 15)
        f2.append(len(s1))
        f3.append(len(s2))

    pnrs['F2'] = f2
    pnrs['F3'] = f3
    
    #Splitting features and labels
    X = pnrs.iloc[:, 3:].values.astype(float)
    y = pnrs.iloc[:, 1].values.astype(float)
    
    # Splitting the dataset into the Training set and Test set
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)
    
    # Feature Scaling
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)


    
    # Fitting Random Forest Classification to the Training set
    from sklearn.ensemble import RandomForestClassifier
    classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
    classifier.fit(X_train, y_train)
    
    # Predicting the Test set results
    y_pred = classifier.predict(X_test)
    
    # Making the Confusion Matrix
    from sklearn.metrics import confusion_matrix
    cm = confusion_matrix(y_test, y_pred)

    return cm  

In [10]:
aclist = []

for i in range(20):
    cm = whole(pnr)
    aclist.append((cm[0][0] + cm[1][1]) / (cm.sum()))
    print((cm[0][0] + cm[1][1]) / (cm.sum()))
    
print(np.mean(aclist))

0.93
0.91
0.97
0.96
0.96
0.94
0.98
0.96
0.95
0.96
0.96
0.93
0.96
0.97
0.96
0.95
0.92
0.94
0.96
0.94
0.9505000000000002


In [ ]:
#using ['ATCT', 'TGGC', 'TGGT', 'CGTG', 'ATC', 'ATCA']

In [7]:
r = res[['ATCT', 'TGGC', 'TGGT', 'CGTG', 'ATC', 'ATCA']].copy()
pnr = pd.concat([pn, r], axis = 1)
pnr = pnr.drop(index = pnr.loc[pnr['MolTyp'] == 'DNA'].index)
pnr = pnr.drop(columns=['Id', 'MolTyp', 'Topol', 'Des'])
pnr = pnr.rename(columns={'ATCT' : 'E1', 'TGGC' : 'E2', 'TGGT' : 'E3', 'CGTG' : 'E4', 'ATC' : 'E5', 'ATCA' : 'E6'})

def whole(pnr):

    ps = pnr.loc[pnr['Gen'] == 1].sample(200)
    ns = pnr.loc[pnr['Gen'] == 0].sample(200)

    pnrs = pd.concat([ps,ns])
    
    
    #feature 1: length
    pnrs['F1'] = [len(i) for i in pnrs['Seq'].to_list()]
    
    f2, f3, f4, f5, f6, f7 = [],[],[],[],[],[]

    for i in pnrs['Seq'].to_list():
        a = SeqRecord(Seq(i))
        (s1, s2) = orf_translator(a, 11, 15)
        f2.append(len(s1))
        f3.append(len(s2))

    pnrs['F2'] = f2
    pnrs['F3'] = f3
    
    #Splitting features and labels
    X = pnrs.iloc[:, 3:].values.astype(float)
    y = pnrs.iloc[:, 1].values.astype(float)
    
    # Splitting the dataset into the Training set and Test set
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)
    
    # Feature Scaling
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)


    
    # Fitting Random Forest Classification to the Training set
    from sklearn.ensemble import RandomForestClassifier
    classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
    classifier.fit(X_train, y_train)
    
    # Predicting the Test set results
    y_pred = classifier.predict(X_test)
    
    # Making the Confusion Matrix
    from sklearn.metrics import confusion_matrix
    cm = confusion_matrix(y_test, y_pred)

    return cm  

In [8]:
aclist = []

for i in range(20):
    cm = whole(pnr)
    aclist.append((cm[0][0] + cm[1][1]) / (cm.sum()))
    print((cm[0][0] + cm[1][1]) / (cm.sum()))
    
print(np.mean(aclist))

0.91
0.99
0.93
0.97
0.94
0.92
0.94
0.97
0.95
0.94
0.94
0.93
0.97
0.95
0.93
0.95
0.96
0.95
0.97
0.94
0.9475


In [11]:
#using ['TGGT','ATC']

In [5]:
r = res[['TGGT','ATC']].copy()
pnr = pd.concat([pn, r], axis = 1)
pnr = pnr.drop(index = pnr.loc[pnr['MolTyp'] == 'DNA'].index)
pnr = pnr.drop(columns=['Id', 'MolTyp', 'Topol', 'Des'])
pnr = pnr.rename(columns={'TGGT' : 'E3', 'ATC' : 'E5'})

def whole(pnr):

    ps = pnr.loc[pnr['Gen'] == 1].sample(200)
    ns = pnr.loc[pnr['Gen'] == 0].sample(200)

    pnrs = pd.concat([ps,ns])
    
    
    #feature 1: length
    pnrs['F1'] = [len(i) for i in pnrs['Seq'].to_list()]
    
    f2, f3, f4, f5, f6, f7 = [],[],[],[],[],[]

    for i in pnrs['Seq'].to_list():
        a = SeqRecord(Seq(i))
        (s1, s2) = orf_translator(a, 11, 15)
        f2.append(len(s1))
        f3.append(len(s2))

    pnrs['F2'] = f2
    pnrs['F3'] = f3
    
    #Splitting features and labels
    X = pnrs.iloc[:, 3:].values.astype(float)
    y = pnrs.iloc[:, 1].values.astype(float)
    
    # Splitting the dataset into the Training set and Test set
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)
    
    # Feature Scaling
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)


    
    # Fitting Random Forest Classification to the Training set
    from sklearn.ensemble import RandomForestClassifier
    classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
    classifier.fit(X_train, y_train)
    
    # Predicting the Test set results
    y_pred = classifier.predict(X_test)
    
    # Making the Confusion Matrix
    from sklearn.metrics import confusion_matrix
    cm = confusion_matrix(y_test, y_pred)

    return cm  

In [6]:
aclist = []

for i in range(20):
    cm = whole(pnr)
    aclist.append((cm[0][0] + cm[1][1]) / (cm.sum()))
    print((cm[0][0] + cm[1][1]) / (cm.sum()))
    
print(np.mean(aclist))

0.95
0.94
0.94
0.95
0.91
0.97
0.94
0.98
0.93
0.94
0.92
0.94
0.92
0.95
0.95
0.96
0.92
0.94
0.88
0.97
0.9400000000000001


In [ ]:
#using ['ATC', 'ATCA', 'GATC', 'ATCT']

In [3]:
r = res[['ATC', 'ATCA', 'GATC', 'ATCT']].copy()
pnr = pd.concat([pn, r], axis = 1)
pnr = pnr.drop(index = pnr.loc[pnr['MolTyp'] == 'DNA'].index)
pnr = pnr.drop(columns=['Id', 'MolTyp', 'Topol', 'Des'])
#pnr = pnr.rename(columns={'TGGT' : 'E3', 'ATC' : 'E5'})

def whole(pnr):

    ps = pnr.loc[pnr['Gen'] == 1].sample(200)
    ns = pnr.loc[pnr['Gen'] == 0].sample(200)

    pnrs = pd.concat([ps,ns])
    
    
    #feature 1: length
    pnrs['F1'] = [len(i) for i in pnrs['Seq'].to_list()]
    
    f2, f3, f4, f5, f6, f7 = [],[],[],[],[],[]

    for i in pnrs['Seq'].to_list():
        a = SeqRecord(Seq(i))
        (s1, s2) = orf_translator(a, 11, 15)
        f2.append(len(s1))
        f3.append(len(s2))

    pnrs['F2'] = f2
    pnrs['F3'] = f3
    
    #Splitting features and labels
    X = pnrs.iloc[:, 3:].values.astype(float)
    y = pnrs.iloc[:, 1].values.astype(float)
    
    # Splitting the dataset into the Training set and Test set
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)
    
    # Feature Scaling
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)


    
    # Fitting Random Forest Classification to the Training set
    from sklearn.ensemble import RandomForestClassifier
    classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
    classifier.fit(X_train, y_train)
    
    # Predicting the Test set results
    y_pred = classifier.predict(X_test)
    
    # Making the Confusion Matrix
    from sklearn.metrics import confusion_matrix
    cm = confusion_matrix(y_test, y_pred)

    return cm  

In [4]:
aclist = []

for i in range(20):
    cm = whole(pnr)
    aclist.append((cm[0][0] + cm[1][1]) / (cm.sum()))
    print((cm[0][0] + cm[1][1]) / (cm.sum()))
    
print(np.mean(aclist))

0.93
0.95
0.95
0.97
0.96
0.94
0.96
0.98
0.98
0.95
0.94
0.96
0.92
0.95
0.97
0.93
0.96
0.94
0.96
0.93
0.9515


In [ ]:
#using ['ATC', 'ATCA', 'ATCT', 'TATC', 'TGGT']

In [17]:
r = res[['ATC', 'ATCA', 'ATCT', 'TATC', 'TGGT']].copy()
pnr = pd.concat([pn, r], axis = 1)
pnr = pnr.drop(index = pnr.loc[pnr['MolTyp'] == 'DNA'].index)
pnr = pnr.drop(columns=['Id', 'MolTyp', 'Topol', 'Des'])
#pnr = pnr.rename(columns={'TGGT' : 'E3', 'ATC' : 'E5'})

def whole(pnr):

    ps = pnr.loc[pnr['Gen'] == 1].sample(200)
    ns = pnr.loc[pnr['Gen'] == 0].sample(200)

    pnrs = pd.concat([ps,ns])
    
    
    f2, f3, f4, f5, f6, f7 = [],[],[],[],[],[]

    for i in pnrs['Seq'].to_list():
        a = SeqRecord(Seq(i))
        (s1, s2) = orf_translator(a, 11, 15)
        f2.append(len(s1))
        f3.append(len(s2))

    pnrs['F2'] = f2
    pnrs['F3'] = f3
    
    #Splitting features and labels
    X = pnrs.iloc[:, 3:].values.astype(float)
    y = pnrs.iloc[:, 1].values.astype(float)
    
    # Splitting the dataset into the Training set and Test set
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)
    
    # Feature Scaling
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)


    
    # Fitting Random Forest Classification to the Training set
    from sklearn.ensemble import RandomForestClassifier
    classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
    classifier.fit(X_train, y_train)
    
    # Predicting the Test set results
    y_pred = classifier.predict(X_test)
    
    # Making the Confusion Matrix
    from sklearn.metrics import confusion_matrix
    cm = confusion_matrix(y_test, y_pred)

    return cm  

In [18]:
aclist = []

for i in range(20):
    cm = whole(pnr)
    aclist.append((cm[0][0] + cm[1][1]) / (cm.sum()))
    print((cm[0][0] + cm[1][1]) / (cm.sum()))
    
print(np.mean(aclist))

0.94
0.95
0.97
0.96
0.97
0.93
0.96
0.94
0.94
0.96
0.95
0.93
0.98
0.96
0.99
0.95
0.97
0.95
0.98
0.95
0.9564999999999999


In [ ]:
#using ['ATC', 'ATCA', 'ATCT']

In [18]:
r = res[['ATC', 'ATCA', 'ATCT']].copy()
pnr = pd.concat([pn, r], axis = 1)
pnr = pnr.drop(index = pnr.loc[pnr['MolTyp'] == 'DNA'].index)
pnr = pnr.drop(columns=['Id', 'MolTyp', 'Topol', 'Des'])
#pnr = pnr.rename(columns={'TGGT' : 'E3', 'ATC' : 'E5'})

def whole(pnr):

    ps = pnr.loc[pnr['Gen'] == 1].sample(200)
    ns = pnr.loc[pnr['Gen'] == 0].sample(200)

    pnrs = pd.concat([ps,ns])
    
    
    #feature 1: length
    pnrs['F1'] = [len(i) for i in pnrs['Seq'].to_list()]
    
    f2, f3, f4, f5, f6, f7 = [],[],[],[],[],[]

    for i in pnrs['Seq'].to_list():
        a = SeqRecord(Seq(i))
        (s1, s2) = orf_translator(a, 11, 15)
        f2.append(len(s1))
        f3.append(len(s2))

    pnrs['F2'] = f2
    pnrs['F3'] = f3
    
    #Splitting features and labels
    X = pnrs.iloc[:, 3:].values.astype(float)
    y = pnrs.iloc[:, 1].values.astype(float)
    
    # Splitting the dataset into the Training set and Test set
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)
    
    # Feature Scaling
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)


    
    # Fitting Random Forest Classification to the Training set
    from sklearn.ensemble import RandomForestClassifier
    classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
    classifier.fit(X_train, y_train)
    
    # Predicting the Test set results
    y_pred = classifier.predict(X_test)
    
    # Making the Confusion Matrix
    from sklearn.metrics import confusion_matrix
    cm = confusion_matrix(y_test, y_pred)

    return cm  

In [19]:
aclist = []

for i in range(20):
    cm = whole(pnr)
    aclist.append((cm[0][0] + cm[1][1]) / (cm.sum()))
    print((cm[0][0] + cm[1][1]) / (cm.sum()))
    
print(np.mean(aclist))

0.97
0.95
0.95
0.98
0.94
0.94
0.93
0.97
0.94
0.93
0.97
0.93
0.99
0.9
0.92
0.93
0.94
0.97
0.91
0.94
0.9450000000000001


In [ ]:
#using ['ATC', 'ATCA']

In [23]:
r = res[['ATC', 'ATCA']].copy()
pnr = pd.concat([pn, r], axis = 1)
pnr = pnr.drop(index = pnr.loc[pnr['MolTyp'] == 'DNA'].index)
pnr = pnr.drop(columns=['Id', 'MolTyp', 'Topol', 'Des'])
#pnr = pnr.rename(columns={'TGGT' : 'E3', 'ATC' : 'E5'})

def whole(pnr):

    ps = pnr.loc[pnr['Gen'] == 1].sample(200)
    ns = pnr.loc[pnr['Gen'] == 0].sample(200)

    pnrs = pd.concat([ps,ns])
    
    
    #feature 1: length
    pnrs['F1'] = [len(i) for i in pnrs['Seq'].to_list()]
    
    f2, f3, f4, f5, f6, f7 = [],[],[],[],[],[]

    for i in pnrs['Seq'].to_list():
        a = SeqRecord(Seq(i))
        (s1, s2) = orf_translator(a, 11, 15)
        f2.append(len(s1))
        f3.append(len(s2))

    pnrs['F2'] = f2
    pnrs['F3'] = f3
    
    #Splitting features and labels
    X = pnrs.iloc[:, 3:].values.astype(float)
    y = pnrs.iloc[:, 1].values.astype(float)
    
    # Splitting the dataset into the Training set and Test set
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)
    
    # Feature Scaling
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)


    
    # Fitting Random Forest Classification to the Training set
    from sklearn.ensemble import RandomForestClassifier
    classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
    classifier.fit(X_train, y_train)
    
    # Predicting the Test set results
    y_pred = classifier.predict(X_test)
    
    # Making the Confusion Matrix
    from sklearn.metrics import confusion_matrix
    cm = confusion_matrix(y_test, y_pred)

    return cm  

In [24]:
aclist = []

for i in range(20):
    cm = whole(pnr)
    aclist.append((cm[0][0] + cm[1][1]) / (cm.sum()))
    print((cm[0][0] + cm[1][1]) / (cm.sum()))
    
print(np.mean(aclist))

0.97
0.92
0.95
0.93
0.92
0.93
0.96
0.96
0.98
0.95
0.92
0.94
0.96
0.99
0.94
0.96
0.9
0.95
0.96
0.97
0.9479999999999998


In [6]:
#using ['GGCA', 'GCCA', 'ACG', 'TGAT', 'TCTA', 'TTAA', 'ATTC', 'G', 'TAAG', 'TTAT']

In [7]:
r = res[['GGCA', 'GCCA', 'ACG', 'TGAT', 'TCTA', 'TTAA', 'ATTC', 'G', 'TAAG', 'TTAT']].copy()
pnr = pd.concat([pn, r], axis = 1)
pnr = pnr.drop(columns=['Id', 'MolTyp', 'Topol', 'Des'])

def whole(pnr):

    ps = pnr.loc[pnr['Gen'] == 1]
    ns = pnr.loc[pnr['Gen'] == 0]

    pnrs = pd.concat([ps,ns])
    
    
    #feature 1: length
    pnrs['F1'] = [len(i) for i in pnrs['Seq'].to_list()]
    
    f2, f3, f4, f5, f6, f7 = [],[],[],[],[],[]

    for i in pnrs['Seq'].to_list():
        a = SeqRecord(Seq(i))
        (s1, s2) = orf_translator(a, 11, 15)
        f2.append(len(s1))
        f3.append(len(s2))

    pnrs['F2'] = f2
    pnrs['F3'] = f3
    
    #Splitting features and labels
    X = pnrs.iloc[:, 3:].values.astype(float)
    y = pnrs.iloc[:, 1].values.astype(float)
    
    # Splitting the dataset into the Training set and Test set
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)
    
    # Feature Scaling
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)


    
    # Fitting Random Forest Classification to the Training set
    from sklearn.ensemble import RandomForestClassifier
    classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
    classifier.fit(X_train, y_train)
    
    # Predicting the Test set results
    y_pred = classifier.predict(X_test)
    
    # Making the Confusion Matrix
    from sklearn.metrics import confusion_matrix
    cm = confusion_matrix(y_test, y_pred)
    scores_mac = cross_val_score(classifier, X, y, cv=5, scoring='f1_macro')
    
    return scores_mac  

In [8]:
aclist = []

for i in range(20):
    fmac = whole(pnr)
    aclist.append(fmac.mean())
    print(fmac.mean(), fmac.std() *2)

print(np.mean(aclist))

0.9201008909653658 0.07213781802967897
0.9201008909653658 0.07213781802967897
0.9201008909653658 0.07213781802967897
0.9201008909653658 0.07213781802967897
0.9201008909653658 0.07213781802967897
0.9201008909653658 0.07213781802967897
0.9201008909653658 0.07213781802967897
0.9201008909653658 0.07213781802967897
0.9201008909653658 0.07213781802967897
0.9201008909653658 0.07213781802967897
0.9201008909653658 0.07213781802967897
0.9201008909653658 0.07213781802967897
0.9201008909653658 0.07213781802967897
0.9201008909653658 0.07213781802967897
0.9201008909653658 0.07213781802967897
0.9201008909653658 0.07213781802967897
0.9201008909653658 0.07213781802967897
0.9201008909653658 0.07213781802967897
0.9201008909653658 0.07213781802967897
0.9201008909653658 0.07213781802967897
0.9201008909653657
